In [1]:
import climetlab as cml
import os
!pip install climetlab climetlab_maelstrom_downscaling

Defaulting to user installation because normal site-packages is not writeable
  Using cached xarray-2022.3.0-py3-none-any.whl (870 kB)


In [8]:
from tensorflow.keras.layers import (Input, Concatenate,Conv3D,LeakyReLU, Dense, Conv2D)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import sigmoid, linear
from tensorflow.keras.preprocessing.image import NumpyArrayIterator
import  tensorflow as tf
tf.config.experimental_run_functions_eagerly(True)
import time
import keras
import tensorflow as tf
import keras.backend as K

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [2]:
cmlds_train = cml.load_dataset("maelstrom-downscaling", dataset="training")
cmlds_val = cml.load_dataset("maelstrom-downscaling", dataset="validation")
cmlds_test = cml.load_dataset("maelstrom-downscaling", dataset="testing")

By downloading data from this dataset, you agree to the terms and conditions defined at https://git.ecmwf.int/projects/MLFET/repos/maelstrom-downscaling-ap5/browse/climetlab-maelstrom-downscaling-ap5/LICENSEIf you do not agree with such terms, do not download the data. 


  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
#auxiliary functions used for parsing the hyerparameters from hparams_dict
def reduce_dict(dict_in: dict, dict_ref: dict):
    """
    Reduces input dictionary to keys from reference dictionary. If the input dictionary lacks some keys, these are 
    copied over from the reference dictionary, i.e. the reference dictionary provides the defaults
    :param dict_in: input dictionary
    :param dict_ref: reference dictionary
    :return: reduced form of input dictionary (with keys complemented from dict_ref if necessary)
    """
    method = reduce_dict.__name__

    # sanity checks
    assert isinstance(dict_in, dict), "%{0}: dict_in must be a dictionary, but is of type {1}"\
                                      .format(method, type(dict_in))
    assert isinstance(dict_ref, dict), "%{0}: dict_ref must be a dictionary, but is of type {1}"\
                                       .format(method, type(dict_ref)) 

    dict_merged = {**dict_ref, **dict_in}
    dict_reduced = {key: dict_merged[key] for key in dict_ref}

    return dict_reduced

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [9]:
#define your hparameters in a dictionary
hparams_dict = {
    "batch_size": 4,
    "lr": 0.001,
    "max_epochs": 5,
    "context_frames": 7,
    "sequence_length": 15,
    "ngf": 16,
    "gan": False, #enable gan
    "enable_wgan":True, #enable wgan 
    "enalbe_cgan":False #enable the conditional information 
}

In [13]:
class WGANModel(object):

    def __init__(self, mode: str = "train", hparams_dict: dict = None,input_shape:list=None, target_shape:list=None): 
        """
         This is a class for building convLSTM GAN architecture by using updated hparameters
             mode                  : string, either "train" or "val" 
             hparams_dict          : dictionary, contains the hyperparameters names and default values
             input_shape           : tf.Tensor shape equal to the input shape
        """
        self.mode = mode
        self.hparams_dict = hparams_dict
        self.hparams = self.parse_hparams()
        self.batch_size = self.hparams.batch_size
        self.learning_rate = self.hparams.lr
        self.max_epochs = self.hparams.max_epochs
        self.sequence_length = self.hparams.sequence_length
        self.context_frames = self.hparams.context_frames
        self.loss_fun = self.hparams.loss_fun
        self.enable_gan = self.hparams.enable_gan
        self.enable_wgan = self.hparams.enable_wgan
        self.enable_cgan = self.hparams.enable_cgan
        self.ngf = self.hparams.ngf
        self.gan = self.hparams.gan
        self.input_shape = input_shape
        self.target_shape = target_shape
        
        
    def hparams_check(self):
        pass
        

    def parse_hparams(self): 
        self.hparams_dict = dotdict(self.hparams_dict)
        return self.hparams_dict

    def generator(self,channels_start=56, z_branch=False):
        """
        Function to build up the generator architecture, here we take UNET as generator
        """

        inputs = Input(input_shape)

        """ encoder """
        s1, e1 = WGANModel.encoder_block(inputs, channels_start, l_large=True)
        s2, e2 = WGANModel.encoder_block(e1, channels_start*2, l_large=False)
        s3, e3 = WGANModel.encoder_block(e2, channels_start*4, l_large=False)

        """ bridge encoder <-> decoder """
        b1 = conv_block(e3, channels_start*8)

        """ decoder """
        d1 = WGANModel.decoder_block(b1, s3, channels_start*4)
        d2 = WGANModel.decoder_block(d1, s2, channels_start*2)
        d3 = WGANModel.decoder_block(d2, s1, channels_start)

        output_temp = Conv2D(1, (1,1), kernel_initializer="he_normal", name="output_temp")(d3)
        if z_branch:
            output_z = Conv2D(1, (1, 1), kernel_initializer="he_normal", name="output_z")(d3)

            model = Model(inputs, [output_temp, output_z], name="t2m_downscaling_unet_with_z")
        else:    
            model = Model(inputs, output_temp, name="t2m_downscaling_unet")

        return model

    @staticmethod
    def conv_block(inputs: tf.Tensor = None, num_filters: int = None, kernel: tuple = (3,3), padding: str="same",
                  activation: str = "relu", kernel_init: str = "he_normal", l_batch_normalization: bool=False): 

        """
        A convolutional layer with optional batch normalization
        :param inputs: the input data with dimensions nx, ny and nc
        :param num_filters: number of filters (output channel dimension)
        :param kernel: tuple indictating kernel size
        :param padding: technique for padding (e.g. "same" or "valid")
        :param activation: activation fuction for neurons (e.g. "relu")
        :param kernel_init: initialization technique (e.g. "he_normal" or "glorot_uniform")
        """
        x = Conv2D(num_filters, kernel, padding=padding, kernel_initializer=kernel_init)(inputs)
        if l_batch_normalization:
            x = BatchNormalization()(x)
        x = Activation(activation)(x)
        return x
    
    @staticmethod
    def conv_block_n(inputs, num_filters, n=2, kernel=(3,3), padding="same", activation="relu", 
                         kernel_init="he_normal", l_batch_normalization=False):
        """
        Sequential application of two convolutional layers (using conv_block).
        """

        x = WGANModel.conv_block(inputs, num_filters, kernel, padding, activation,
                       kernel_init, l_batch_normalization)
        for i in np.arange(n-1):
            x = WGANModel.conv_block(x, num_filters, kernel, padding, activation,
                           kernel_init, l_batch_normalization)
        return x
    
    @staticmethod
    def encoder_block(inputs, num_filters, kernel_maxpool: tuple=(2,2), l_large: bool=False):
        """
        One complete encoder-block used in U-net
        """
        if l_large:
            x = WGANModel.conv_block_n(inputs, num_filters, n=2)
        else:
            x = WGANModel.conv_block(inputs, num_filters)

        p = MaxPool2D(kernel_maxpool)(x)

        return x, p
    
    @staticmethod
    def decoder_block(inputs, skip_features, num_filters, kernel: tuple=(3,3), strides_up: int=2, padding: str= "same",
                      activation: str="relu", kernel_init: str="he_normal", l_batch_normalization: bool=False):
        """
        One complete decoder block used in U-net (reverting the encoder)
        """

        x = Conv2DTranspose(num_filters, (strides_up, strides_up), strides=strides_up, padding="same")(inputs)
        x = Concatenate()([x, skip_features])
        x = WGANModel.conv_block_n(x, num_filters, 2, kernel, padding, activation, kernel_init, l_batch_normalization)
        return x
    
    def discriminator(self):
        """
        Discriminator: this discriminaotr so far perfoms best on the precipitation dataset
        """
        
        x = Input(self.target_shape)
        conv1 = Conv3D(filters=4, kernel_size=2, strides=(1, 1, 1), padding='same')(x)
        conv1 = Activation("relu")(conv1)
        conv2 = tf.reshape(conv1, [-1,1])
        fc2 = LeakyReLU(0.2)(conv2)
        out_logit = Dense(1)(fc2)
        out = tf.nn.sigmoid(out_logit) 
        D = Model(x, [out,out_logit])
        return D


    def define_optimizers(self):
        self.d_optim = tf.keras.optimizers.Adam(self.learning_rate)
        self.g_optim = tf.keras.optimizers.Adam(self.learning_rate)
        
    def get_gen_loss(self):
        """
        Define generator loss

        Return:  the loss of generator given inputs
        """
        real_labels = tf.ones_like(self.D_fake)
        self.G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D_fake_logits, labels=real_labels))            
        
        return self.G_loss
    

    def get_disc_loss(self):
        """
        Return the loss of discriminator given inputs
        """
        real_labels = tf.ones_like(self.D_real)
        gen_labels = tf.zeros_like(self.D_fake)
        self.D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D_real_logits,
                                                                                  labels=real_labels))
        self.D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.D_fake_logits,
                                                                                  labels=gen_labels))
        self.D_loss = self.D_loss_real + self.D_loss_fake

        return self.D_loss
 

    def get_recon_loss(self, target, gen_images):
        recon_loss = tf.reduce_mean(tf.square(target - gen_images))
        return recon_loss

    @tf.function     
    def train_step(self, inputs, target, i):
        """
        Training models
        """
        self.G = self.generator()
        self.D = self.discriminator()
        self.define_optimizers()
        with tf.GradientTape() as d_tape, tf.GradientTape() as g_tape:

            gen_images = self.G(inputs)
            self.D_real, self.D_real_logits = self.D(target)
            self.D_fake, self.D_fake_logits = self.D(gen_images[:,self.context_frames-1:,:,:,0]) 

            g_loss = self.get_gen_loss()
            d_loss = self.get_disc_loss()
            recon_loss = self.get_recon_loss(target[:, :, :, :, 0], gen_images[:,self.context_frames-1:,:,:,0])
            d_gradients = d_tape.gradient(d_loss, self.D.trainable_variables)

            if not self.gan:
                if i == 0:
                    print("You are only training generator")
                # if the discriminator is not used for training, only train generator part
                g_gradients = g_tape.gradient(recon_loss, self.G.trainable_variables)
                self.g_optim.apply_gradients(zip(g_gradients, self.G.trainable_variables))
            else:
                if i == 0:
                    print("You are training both generator and discriminator")
                g_gradients = g_tape.gradient(g_loss + recon_loss, self.G.trainable_variables)
                self.d_optim.apply_gradients(zip(d_gradients, self.D.trainable_variables))
                self.g_optim.apply_gradients(zip(g_gradients, self.G.trainable_variables))

        return g_loss, d_loss, recon_loss


    def calculate_samples(self,predictors,train_ratio=0.5):
        """
        calculate the number of training and validatioin samples 
        """
        self.Itrain = range(int(predictors.shape[0]*train_ratio))
        self.Ieval = range(int(predictors.shape[0]*train_ratio), predictors.shape[0])
        self.train_samples = int(predictors.shape[0]*train_ratio)
        self.val_samples = predictors.shape[0] - int(predictors.shape[0]*train_ratio)
        print("Total samples: {}, trainng samples: {}".format(self.train_samples + self.val_samples,self.train_samples))


    def make_data_generator(self, predictors, target):
        self.calculate_samples(predictors)
        train_predictors, train_target = predictors[Itrain, ...], target[Itrain, ...]
        val_predictors, val_target = predictors[Ieval, ...], target[Ieval, ...]
        train_dataset = tf.data.Dataset.from_tensor_slices((train_predictors,train_target))
        val_dataset = tf.data.Dataset.from_tensor_slices((val_predictors,val_target))
        train_dataset = train_dataset.shuffle(100).repeat(self.max_epochs).batch(self.batch_size)
        val_dataset = val_dataset.batch(self.batch_size)
        self.train_iterator = iter(train_dataset)
        self.val_iterator = iter(val_dataset) 

    def train(self,log_freq=5):

        iterations_epoch = self.train_samples // self.batch_size
        iteration = self.max_epochs * iterations_epoch

        for step in range(iteration):
            (x,y) = next(self.train_iterator)
            
            train_start_time = time.time()
            g_loss, d_loss, recon_loss = self.train_step(x, y, step)
            train_step_time = time.time() - train_start_time

            if step % log_freq == 0:
                template = '[{}/{}] D_loss={:.5f} G_loss={:.5f}, g_recon_loss={:.5f} training time per step: {:.5f}/s'
                print(template.format(step, iteration, d_loss,g_loss,recon_loss, train_step_time))


    def prediction(self):
        iterations = self.val_samples // self.batch_size
        (x_val,y_val) = next(self.val_iterator)
        is_first = True
        for i in range(iterations):
            output = modelCase.G(x_val)
            if is_first:
                outputs = output
                is_first = False
            else:
                outputs = np.concatenate((outputs,output), axis=0)
        print("Inference is done")
        return outputs

modelCase = WGANModel(mode="train", hparams_dict=hparams_dict,
                                         input_shape=[7,128,128,7],target_shape=[8,128,128,1])
modelCase.make_data_generator(predictors, target_scaled)
modelCase.train()

IndentationError: expected an indented block (2858739789.py, line 31)